
<!-- 
This work includes material from Apache Spark documentation licensed under Apache License 2.0. 
Original content available at: https://spark.apache.org/documentation.html 

This document contains modified excerpts from Apache Spark documentation, originally licensed under Apache License 2.0. Modifications made by Krzysztof Płatek.

-->

# Co to jest Spark?

* framework do rozproszonego przetwarzania dużych zbiorów danych
* dane są przetwarzane w pamięci operacyjnej
* wbudowany optymalizator
* obsługuje różne źródła danych
  * JDBC
  * pliki: JSON, Avro, Parquet, Delta, Iceberg, ORC, CSV pobierane z różnych zasobów dyskowych:
    * HDFS, S3, ADLS, GCS
  * Cassandra
  * MongoDB
  * ...
* napisany w języku Scala
  * co implikuje uruchomienie na maszynie wirtualnej Java'y (JVM)
* wsparcie dla Python (PySpark), SQL, Scala, Java, R
* przetwarzania wsadowe i strumieniowe
* architektura master-worker

Darmowa książka:
1. https://pages.databricks.com/rs/094-YMS-629/images/LearningSpark2.0.pdf

Docs:
1. https://spark.apache.org/docs/latest/api/python/index.html
2. https://spark.apache.org/docs/latest/index.html 



# Komponenty aplikacji Spark'owej

1. Aplikacja (*application*) - program napisany za pomocą Spark'a. Składa się z dwóch rodzajów komponentów:
  * `driver` - "zarządza" aplikacją. Jest tylko jeden.
  * `executor` - odpowiedzialny za wykonanie obliczeń. Może być ich wiele.
2. `SparkSession` (sesja Spark) - obiekt, za pomocą którego korzystamy z możliwości Spark'a
3. `Job` - wykonywane lub wykonane przetwarzanie zlecone Spark'owi
4. `Stage` - `Job` wykonywane jest w etapach
5. `Task` - obliczenia w ramach `Stage` wykonywane są w `Task`'ach


## Uproszczona architektura aplikacji Spark

![cluster-overview](https://raw.githubusercontent.com/chrispi21/spark-postgraduate-studies/main/resources/cluster-overview.png)

## Managerowie klastra (`cluster managers`)

Manager klastra dpowiada za przydział zasobów obliczeniowych:
1. `Standalone`
2. `Mesos` - przestarzały
3. `YARN`
4. `Kubernetes`


## SparkSession

Obiekt `SparkSession` jest dostępny w notebook'u pod nazwą `spark`:

In [0]:
spark

## Job vs Stage vs Task

Web UI demo

In [0]:
# Job #1
spark.sql("select 1").collect()

In [0]:
# Job #2
spark.conf.set("spark.sql.cbo.enabled", "false")
spark.conf.set("spark.sql.adaptive.enabled", "false")

df1 = spark.range(1000000).selectExpr("(rand() * 100) % 10 as mod10").groupby("mod10").count().orderBy("mod10")
df2 = spark.range(20000000).selectExpr("id % 20 as mod20").groupby("mod20").count().orderBy("mod20")
display(
  df1.join(df2, on=(df1["mod10"] == df2["mod20"]))
)

spark.conf.set("spark.sql.cbo.enabled", "true")
spark.conf.set("spark.sql.adaptive.enabled", "true")


# Jak Spark przechowuje dane w pamięci?

Podział danych na partycje:

In [0]:
# w tym przykładzie wymuszamy ręcznie podział danych na 32 partycje - standardowo Spark robi to automatycznie
df = spark.range(1000, numPartitions=32)
df.rdd.getNumPartitions()

Zobaczmy jak wpływa to na liczbę tasków?

In [0]:
df.count()

# Leniwe wykonanie kodu (`lazy evaluation`)

Co to jest `DataFrame` w Spark'u?

In [0]:
df1 = spark.sql("select 1 as id")

In [0]:
df2 = spark.sql("select 2 as id")


In [0]:
df3 = df1.union(df2)

Co się dzieje?

Akcja (`action`) vs. Transformacja (`transformation`)?

In [0]:
df3.collect()

# RDD

RDD jest abstrakcją, na której oparte są `DataFrame`'y. Skupmy się wyłącznie na ważnych właściwościach:

* Resilient - odporność na awarię - możliwość ponownego przeliczenia części transformacji bez konieczności przeliczania całości
* Distributed - rozproszony - dane są przechowywane na wielu węzłach
* Dataset - zbiór danych

RDD też są wykonywane leniwie, ale nie mają właściwości `DataFrame`, jeśli chodzi o optymalizację.

# Wąskie i szerokie transformacje (`narrow and wide transformations`)

1. Wąskie - jedna partycja wyjściowa powstaje na bazie jednej partycji wejściowej. Przykłady: filtrowanie, projekcja
2. Szerokie - jedna partycja wyjściowa powstaje na bazie wielu partycji wejściowych. Przykłady: grupowanie, sortowanie, łączenie

Transformacje wąskie nie wymagają przesyłu danych po sieci.

Transformacje szerokie wymagają wymiany danych między executorami (tzw. `shuffle`).

Porównajmy czas wykonania i metryki związane z poniższymi przykładami:


In [0]:
df1 = spark.range(10_000_000)
df2 = df1.selectExpr("1000*rand() % 100 as mod100")
df2.take(1000)
df2.count()

In [0]:
df1 = spark.range(10_000_000)
df2 = df1.selectExpr("1000*rand() % 100 as mod100").groupBy("mod100").count().orderBy("mod100")
df2.take(10)

# Serializacja i deserializacja (`serialization`, `deserialization`)

W skrócie: SerDe

Są to operacje, które są niezbędne do sieciowego przesłania danych. Mają ogromne znaczenie dla wydajności przy wymianie danych między executorami (`shuffle`).

Spark korzysta z własnej reprezentacji danych w pamięci, która jest zoptymalizowana pod kątem szybkiej serializacji i deserializacji.

## Serializacja

Aby przesłać dane przez sieć obiekt musi zostać zamieniony na postać binarną. Mówimy wtedy, że obiekt został zserializowany.

<br>

```
obiekt -> postać binarna
```

## Deserializacja
Odbiorca wiadomości odczytuje dane przesłane siecią w postaci binarnej i zamienia na obiekt:

<br>

```
postać binarna -> obiekt
```

# Encoders

W poprzedniej sekcji padło stwierdzenie:

"Spark korzysta z własnej reprezentacji danych w pamięci, która jest zoptymalizowana pod kątem szybkiej serializacji i deserializacji."

Jak to się ma to reprezentacji JVM?

Tutaj również ma miejsce proces serializacji i deserializacji, który obsługuje wewnętrzny mechanizm enkoderów (`encoders`, `dataset encoders`). Najczęściej możemy uniknąć SerDe, korzystając z funkcji wbudowanych w Spark'a albo wyrażeń SQL.

In [0]:
# funkcja wbudowana - unikamy SerDe
from pyspark.sql.functions import pmod

spark.range(100).select(pmod("id", 10)).explain(extended=True)

In [0]:
%sql
-- SQL - unikamy Serde
EXPLAIN EXTENDED
  SELECT mod(id, 10)
  FROM RANGE(1000)

In [0]:
%scala
// Pojawia się DeserializeToObject - SerDe: Java <-> Spark
spark.range(1000).map(id => id % 10).explain(extended=true)

In [0]:
%python
# Tak nie robimy - są lepsze sposoby na własne funkcje w PySpark!

# Pojawia się BatchEvalPython
# Po sprawdzeniu:
# https://github.com/apache/spark/blob/v3.5.0/sql/core/src/main/scala/org/apache/spark/sql/execution/python/BatchEvalPythonExec.scala#L103
# Oznacza to SerDe: interpreter Python <-> JVM oraz JVM <-> Spark
from pyspark.sql.functions import udf
mod10 = udf(lambda x: x % 10)
spark.range(1000).select(mod10("id")).explain(extended=True)

# Catalyst

Optymalizator kosztowy wbudowany w Spark'a (CBO - `Cost Based Optimizer`).




## Fazy optymalizacji zapytania

### Analiza

Sprawdzana jest poprawność składniowa oraz poprawność nazw tabel i kolumn.

In [0]:
%scala
// Błąd
spark.sql("select ")

In [0]:
%scala
// Błąd
spark.sql("select 1 as col1").where("colXYZ = 5")

### Logiczna optymalizacja

W tym momemcie generowane są alternatywne plany zapytań uwzględniając szereg reguł optymalizacyjnych (optymalizator regułowy).

In [0]:
# interesuje nas sekcja: == Optimized Logical Plan ==

spark.sql("select 1 as col1").where("col1 = 5").explain(extended=True)

### Plan fizyczny

Generowane są plany fizyczne na bazie planu logicznego. Każdy plan ma obliczany koszt z nim związany. Wybierany jest ten z najniższym kosztem.

In [0]:
# interesuje nas sekcja: == Physical Plan ==

spark.sql("select 1 as col1").where("col1 = 5").explain(extended=True)

### Generowanie kodu

Na bazie optymalnego planu fizycznego generowany jest kod bajtowy (`bytecode`). Dlaczego? Plan fizyczny jest skomplikowany tj. jego wykonanie wymagałoby tworzenia wielu nowych obiektów w pamięci. Ten etap ma na celu wygenerowanie kodu, który jest optymalny pod kątem wykonania na maszynie wirtualnej Java'y (JVM). Wiele tego typu optymalizacji (i nie tylko) obejmuje [Project Tungsten](https://www.databricks.com/blog/2015/04/28/project-tungsten-bringing-spark-closer-to-bare-metal.html).

In [0]:
spark.sql("select 1 as col1").where("col1 = 5").explain(mode="codegen")

In [0]:
spark.range(1000).explain(mode="codegen")

# Adaptive Query Execution (AQE)

Domyślnie Spark optymalizuje plan wykonania bazując na statystykach już wykonanych etapów. 

Więcej szczegółów:

https://www.databricks.com/blog/2020/05/29/adaptive-query-execution-speeding-up-spark-sql-at-runtime.html

In [0]:
# Przykład: SQL/Dataframe

df1 = spark.range(1000000).selectExpr("(rand() * 100) % 10 as mod10").groupby("mod10").count().orderBy("mod10")
df2 = spark.range(20000000).selectExpr("id % 20 as mod20").groupby("mod20").count().orderBy("mod20")
display(
  df1.join(df2, on=(df1["mod10"] == df2["mod20"]))
)